In [1]:
print("hiliii")

hiliii


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
# LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
# LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [4]:

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [45]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest")

E0000 00:00:1759390043.082229 1597511 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759390043.084569 1597511 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [24]:
print(llm.invoke("Hello, my name is LangGraph. How can I help you today?").content)

Hello LangGraph! That's a fascinating name. I'm an AI assistant here to help you with information, creative tasks, or anything else you might need.

Since you're named LangGraph, I'm curious: **Are you related to the LangGraph library for building stateful, multi-agent applications?**

Regardless, how can I assist *you* today? Do you have a question, a task, or are you just looking to chat?


In [26]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hi there! How can I help you today?
Today's date is **October 26, 2023**.
goodbye take care yourself


In [27]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [28]:
store={}

In [29]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [30]:

config = {"configurable": {"session_id": "firstchat"}}

In [31]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [33]:

model_with_memory.invoke(("Hi! I'm Areeb"),config=config).content

"Hi Areeb! It's nice to meet you. What brings you here today?"

In [34]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Based on our conversation, you told me your name is **Areeb**.'

In [46]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the py files from source directory

loader = DirectoryLoader('./data', glob="./*.py", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [47]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [48]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [49]:
question ="give me summary of code file "
print(retrieval_chain.invoke(question))

The provided context snippets from the code file `data/hello2.py` show fragments of Python code involving `print(a)`, `break`, and `else` blocks, suggesting a loop structure where some condition might be checked, leading to a potential break or execution of an `else` block that modifies a variable `st` (`st=st[:-1]`). There are also commented-out lines referencing `ans=str(a)`.
